In [44]:
import os
import time
import torch
import torch as tc
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable
import matplotlib.pyplot as plt
#get_ipython().magic('matplotlib inline')
from PIL import Image
import numpy as np
import sys,os
import math
import gc


In [45]:
import torch.backends.cudnn as cudnn
import torch
import sys
print('__Python VERSION:', sys.version)
print('__pyTorch VERSION:', torch.__version__)
print('__CUDA VERSION')
#from subprocess import call
#call(["nvcc", "--version"])
print('__CUDNN VERSION:', torch.backends.cudnn.version())
print('__Number CUDA Devices:', torch.cuda.device_count())
print('__Devices')
#call(["nvidia-smi", "--format=csv", "--query-gpu=index,name,driver_version,memory.total,memory.used,memory.free"])
print('Active CUDA Device: GPU', torch.cuda.current_device())

__Python VERSION: 3.5.3 |Anaconda 4.4.0 (64-bit)| (default, Mar  6 2017, 11:58:13) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
__pyTorch VERSION: 0.2.0_4
__CUDA VERSION
__CUDNN VERSION: 6021
__Number CUDA Devices: 1
__Devices
Active CUDA Device: GPU 0


In [46]:
cudnn.benchmark = True

use_gpu = torch.cuda.is_available()
#use_gpu = False
if use_gpu:
	print("You are using CUDA. If it is not what you want, manually set this as False!")

You are using CUDA. If it is not what you want, manually set this as False!


In [47]:
nc = 3
ngpu = 1
nz = 100
ngf = 64
ndf = 64
n_extra_d = 0
n_extra_g = 1 # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
imageSize = 64


In [48]:
# This should, in the future, be set in CLI
chosen_dataset = 'CIFAR10'

datasets = {
    'MNIST': torchvision.datasets.MNIST,
    'CIFAR10': torchvision.datasets.CIFAR10,
    'ANIME': '/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
    #'FashionMNIST': torchvision.datasets.FashionMNIST
}

dataset = datasets[chosen_dataset]

In [49]:
print(dataset)

<class 'torchvision.datasets.cifar.CIFAR10'>


In [88]:
possible_parameters = {
    'MNIST': {
        'ndf': 64,
        'ngf': 64,
        'nz': 50,
        'nc': 1,
        'n_classes' : 10,
    },
    'CIFAR10': {
        'ndf': 64,
        'ngf': 64,
        'nz': 100,
        'nc': 3,
        'n_extra_d' : 0,
        'n_extra_g' : 1, # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
        'imageSize' : 32,
        'n_classes' : 10
    },
    'ANIME': {
        'nc' : 3,
        'ngpu' : 1,
        'nz' : 100,
        'ngf' : 64,
        'ndf' : 64,
        'n_extra_d' : 0,
        'n_extra_g' : 1, # Aqui a jogada é que o gerador deve ser mais poderoso q o detetive
        'imageSize' : 64,
        'n_classes' : 1


    }
    #'FashionMNIST': {}
}

In [89]:
ngf = possible_parameters[chosen_dataset]['ngf']
ndf = possible_parameters[chosen_dataset]['ndf']
code_size = possible_parameters[chosen_dataset]['nz']
n_channels = possible_parameters[chosen_dataset]['nc']
imageSize = possible_parameters[chosen_dataset]['imageSize']
n_classes = possible_parameters[chosen_dataset]['n_classes']

## Creating the Dataset!

In [ ]:
dataset_done = dataset('./datasets', train=True, download=True, transform=transform)

In [61]:
batch_size=64


In [58]:
if dataset == 'ANIME':
    dataset = dset.ImageFolder(
    root='/home/gabriel/Redes Neurais/Projeto_Final_GANS/Tutorial_2/dataset/min_anime-faces',
    transform=transforms.Compose([
            transforms.Scale((imageSize, imageSize)),
            # transforms.CenterCrop(opt.imageSize),
            transforms.ToTensor(),
            #transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
        ])
    )
else:
    transform=transforms.Compose([
                transforms.Scale((imageSize, imageSize)),
                transforms.ToTensor(),
                transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)), # bring images to (-1,1)
            ]) 
    dataset_done = dataset('./datasets', train=True, download=True, transform=transform)
    dataloader = tc.utils.data.DataLoader(dataset_done, batch_size=batch_size, shuffle=True, num_workers=1
                                          )


## Criando os modelos

In [67]:
class _netD_1(nn.Module):
	def __init__(self, ngpu, nz, nc, ndf,  n_extra_layers_d):
		super(_netD_1, self).__init__()
		self.ngpu = ngpu
		main = nn.Sequential(
			# input is (nc) x 96 x 96
			nn.Conv2d(nc, ndf, 4, 2, 1, bias=False), # 5,3,1 for 96x96
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf) x 32 x 32
			nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(ndf * 2),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*2) x 16 x 16
			nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(ndf * 4),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*4) x 8 x 8
			nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
			nn.BatchNorm2d(ndf * 8),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ndf*8) x 4 x 4
		)

		# Extra layers
		for t in range(n_extra_layers_d):
			main.add_module('extra-layers-{0}.{1}.conv'.format(t, ndf * 8),
							nn.Conv2d(ndf * 8, ndf * 8, 3, 1, 1, bias=False))
			main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ndf * 8),
							nn.BatchNorm2d(ndf * 8))
			main.add_module('extra-layers-{0}.{1}.relu'.format(t, ndf * 8),
							nn.LeakyReLU(0.2, inplace=True))


		main.add_module('final_layers.conv', nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False))
		main.add_module('final_layers.sigmoid', nn.Sigmoid())
		# state size. 1 x 1 x 1
		self.main = main

	def forward(self, input):
		gpu_ids = None
		if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
			gpu_ids = range(self.ngpu)
		output = nn.parallel.data_parallel(self.main, input, gpu_ids)
		return output.view(-1, 1)

In [68]:
class _netG_1(nn.Module):
	def __init__(self, ngpu, nz, nc , ngf, n_extra_layers_g):
		super(_netG_1, self).__init__()
		self.ngpu = ngpu
		#self.nz = nz
		#self.nc = nc
		#self.ngf = ngf
		main = nn.Sequential(
			# input is Z, going into a convolution
			# state size. nz x 1 x 1
			nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
			nn.BatchNorm2d(ngf * 8),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ngf*8) x 4 x 4
			nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
			nn.BatchNorm2d(ngf * 4),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ngf*4) x 8 x 8
			nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
			nn.BatchNorm2d(ngf * 2),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ngf*2) x 16 x 16
			nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
			nn.BatchNorm2d(ngf),
			nn.LeakyReLU(0.2, inplace=True),
			# state size. (ngf) x 32 x 32
		)

		# Extra layers
		for t in range(n_extra_layers_g):
			main.add_module('extra-layers-{0}.{1}.conv'.format(t, ngf),
							nn.Conv2d(ngf, ngf, 3, 1, 1, bias=False))
			main.add_module('extra-layers-{0}.{1}.batchnorm'.format(t, ngf),
							nn.BatchNorm2d(ngf))
			main.add_module('extra-layers-{0}.{1}.relu'.format(t, ngf),
							nn.LeakyReLU(0.2, inplace=True))

		main.add_module('final_layer.deconv', 
						 nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False)) # 5,3,1 for 96x96
		main.add_module('final_layer.tanh', 
						 nn.Tanh())
			# state size. (nc) x 96 x 96

		self.main = main


	def forward(self, input):
		gpu_ids = None
		if isinstance(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
			gpu_ids = range(self.ngpu)
		return nn.parallel.data_parallel(self.main, input, gpu_ids), 0

In [70]:

netG = _netG_1(ngpu, nz, nc, ngf, n_extra_g)
#netG_parallel = torch.nn.DataParallel(_netG_1(ngpu, nz, nc, ngf, n_extra_g))

netD = _netD_1(ngpu, nz, nc, ndf, n_extra_d)
#netD_parallel = torch.nn.DataParallel(_netD_1(ngpu, nz, nc, ndf, n_extra_d))

## Inicializador de pesos

In [71]:
def weights_init(m):
	classname = m.__class__.__name__
	if classname.find('Conv') != -1:
		m.weight.data.normal_(0.0, 0.02)
	elif classname.find('BatchNorm') != -1:
		m.weight.data.normal_(1.0, 0.02)
		m.bias.data.fill_(0)

In [74]:
netG.apply(weights_init)
#netG_parallel.apply(weights_init)
netD.apply(weights_init)
#netD_parallel.apply(weights_init)
print()

## Losses

In [75]:
criterion = nn.BCELoss()
criterion_MSE = nn.MSELoss()

## Sizes of the tensors

In [76]:
input = torch.FloatTensor(batch_size, 3, imageSize, imageSize)
print(input.size())
noise = torch.FloatTensor(batch_size, nz, 1, 1)
print(noise.size())

torch.Size([64, 3, 32, 32])
torch.Size([64, 100, 1, 1])


In [82]:
max(dataloader.dataset.train_labels)


9

In [83]:
binary=False
#Ele testa pergunta se vc quer que o seu Z venha da distribuição bernoulli
if binary:
	bernoulli_prob = torch.FloatTensor(batch_size, nz, 1, 1).fill_(0.5)
	fixed_noise = torch.bernoulli(bernoulli_prob)
else:
	fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)

In [90]:
label = torch.FloatTensor(batch_size,n_classes)
print(label.size())
real_label = 1
fake_label = 0


torch.Size([64, 10])


## Broadcast then to the GPU

In [94]:
if use_gpu:
	netD.cuda()
	netG.cuda()
	#netD_parallel.cuda()
	#netG_parallel.cuda()
	criterion = criterion.cuda()
	criterion_MSE = criterion_MSE.cuda()
	input,label = input.cuda(), label.cuda()
	noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

## Turn then on Variables

In [96]:
input = Variable(input)
label = Variable(label)
noise = Variable(noise)
fixed_noise = Variable(fixed_noise)

RuntimeError: Variable data has to be a tensor, but got Variable

## Parâmetros de treinamento

In [99]:
beta1, beta2 = 0.9,0.999
lr = 2.0e-4
optimizerD = optim.Adam(netD.parameters(), lr = lr, betas = (beta1, beta2))
#optimizerD = optim.Adam(netD_parallel.parameters(), lr = lr, betas = (beta1, beta2))

optimizerG = optim.Adam(netG.parameters(), lr = lr, betas = (beta1, beta2))
#optimizerG = optim.Adam(netG_parallel.parameters(), lr = lr, betas = (beta1, beta2))


## Output_dir

In [ ]:

outputDir = 'outputdir_train_sobrescreve'

try:
	os.makedirs(outputDir)
except OSError as err:
	print("OS error: {0}".format(err))

In [91]:
for k,data in enumerate(dataloader):
    if k>0:
        break
    inp,label = data
    print(type(inp))
    print(inp.size())
    print(type(label))
    print(label.size())
    print(label.max())

<class 'torch.FloatTensor'>
torch.Size([64, 3, 32, 32])
<class 'torch.LongTensor'>
torch.Size([64])
9
